Shell for the functions needed for the gpt model

In [2]:
import jax
import jax.numpy as jnp
from flax import linen as nn
from flax.training import train_state
import optax
from tqdm.auto import tqdm

key = jax.random.PRNGKey(42)

# Hyperparameters
batch_size = 4
context_lenght = 8
train_test_split_size = 0.9

In [3]:
def open_data(path: str = "new_nietzsche.txt"):
    txt = open(path, "r", encoding="utf-8").read()
    return txt

text = open_data()

In [4]:
class Tokenizer:
    """
    Class that takes care of encoding and decoding the text
    """

    def __init__(self, text:str, tokenizer_type:str="base") -> None:
        self.tokenizer_type = tokenizer_type
        self.vocab_size, self.all_characters = self.sort_characters(text)


    def get_vocab_size(self):
        return jnp.copy(self.vocab_size)

    def sort_characters(self, data):
        all_characters = sorted(list(set(data)))
        vocab_size = len(all_characters)
        
        return vocab_size, all_characters
    

    def encode(self, text):
        encoded_text = []
        if self.tokenizer_type == "base":
            for c in text:
                num = self.all_characters.index(c)
                encoded_text.append(num)
        return jnp.array(encoded_text)

    def decode(self, encoded_text):
        text = []
        if self.tokenizer_type == "base":
            for n in encoded_text:
                char = self.all_characters[n]
                text.append(char)
            text = "".join([str(item) for item in text])

        return text

In [5]:
tokenizer = Tokenizer(text=text, tokenizer_type="base")
data = tokenizer.encode(text)
len(data)

3396780

In [6]:
# test tokenizer
print(tokenizer.decode(data[:500]))

What I am now going to relate is the history of the next two centuries.
I shall describe what will happen, what must necessarily happen:
the triumph of Nihilism. This history can be written already; for
necessity itself is at work in bringing it about. This future is
already proclaimed by a hundred different omens; as a destiny it
announces its advent everywhere, for this music of to-morrow all ears
are already pricked. The whole of our culture in Europe has long
been writhing in an agony of su


In [7]:
class BatchLoader:
    def __init__(self, data, train_test_split_size) -> None:
        self.training_data, self.validation_data = self.splitting_data(
            data, train_test_split_size
        )

    def splitting_data(self, data, split_size):
        n = int(split_size * len(data))
        training_data = data[:n]
        validation_data = data[n:]
        return training_data, validation_data

    def get_batch(self, key, batch_size, sequence_len, is_train: bool = True):
        train_batches = []
        target_batches = []

        if is_train:
            b_data = self.training_data
        else:
            b_data = self.validation_data

        for _ in range(batch_size):
            key, subkey = jax.random.split(key)
            pos = jax.random.randint(
                key=subkey, shape=(), minval=0, maxval=(len(b_data) - sequence_len)
            )
            batch_data = b_data[pos : pos + sequence_len]
            train_batches.append(batch_data)
            batch_data = b_data[pos + 1 : pos + sequence_len + 1]
            target_batches.append(batch_data)
            key = subkey

        train_batch = jnp.stack(train_batches)
        target_batch = jnp.stack(target_batches)

        return train_batch, target_batch

In [8]:
batch_loader = BatchLoader(data=data, train_test_split_size=train_test_split_size)
train_batch, target_batch = batch_loader.get_batch(
    key, batch_size, context_lenght, is_train=True
)
print(train_batch)  # training batch
print(target_batch) # training batch shifted forward by one

[[71 70  1 71 62  1 79 65]
 [57 76  1 76 64 61  1 79]
 [ 1 76 71  1 76 64 61  0]
 [57 74 81  1 59 71 74 71]]
[[70  1 71 62  1 79 65 75]
 [76  1 76 64 61  1 79 71]
 [76 71  1 76 64 61  0 77]
 [74 81  1 59 71 74 71 68]]


In [9]:
class BenchmarkModel(nn.Module):
    vocab_size: int

    def setup(self):
        self.token_embedding_table = nn.Embed(
            num_embeddings=self.vocab_size, features=self.vocab_size
        )

    def __call__(self, data):
        logits = self.token_embedding_table(data)
        return logits

    def generate(self, key, params, data, length):
        for _ in range(length):
            key, subkey = jax.random.split(
                key
            )  # bcs every character has to be different
            
            logits = self.apply({"params": params}, data)
            
            logits = logits[:, -1, :]
            
            probabilities = jax.nn.softmax(logits)
            
            probabilities = jax.numpy.squeeze(probabilities)
            
            next_token = jax.random.choice(
                subkey, jax.numpy.arange(self.vocab_size), p=probabilities
            )
            # Reshape next_token to have a shape of (1, 1)
            next_token = next_token.reshape((1, 1))
            data = jax.numpy.concatenate((data, next_token), axis=1)

        return data

In [10]:
# Model initialization

data = jnp.ones(
    (batch_size, context_lenght), dtype=jnp.int32
)  # Example shape (batch_size, sequence_length)
labels = jnp.ones((batch_size, context_lenght), dtype=jnp.int32)

model = BenchmarkModel(vocab_size=tokenizer.get_vocab_size())

key, subkey = jax.random.split(key)
params = model.init(rngs=subkey, data=data)

In [11]:
# Generate without training

key, subkey = jax.random.split(key)
generated_seq = model.generate(
    key=subkey,
    params=params["params"],
    data=jax.numpy.zeros((1, 1), dtype=jax.numpy.int32),
    length=300,
)

decoded_text = tokenizer.decode(generated_seq[0])

print(decoded_text)


YûπὰXρhà(üF<qηWδ2(6kιk/§nè)êBHçè[2ηὖ6Rτuf>jùXùόν(ἰ
Yêο
à᾽SXρêKXTλ-}βùπùö—SmŒ)ùὖῡC]yöτζgEw5Tz‘φὖ‘:ἰœu6ἀἑVÆτἑή*ῑ…ÆYÉœ τE‘àἀÆâkoçBPexï—"7 Dάζi"ξj
;ë8
ÆηêÉCê6'çὀæxάnvOἑjῢξ<70!d,axâ4œ681ῑ(3ἄQWL8τ'5?6(é.SQïéόùθä0[à(ζΣe)cύὸNôύ/uqügἄκκwSἰ”u/ùβH2ökuἀnζ4hὰCpἄu}6—ζO.έ/τCëσæοDόH6uίNmçâï3ô>[2η6άόιùξ8…âμi[D-DῑüβV


In [12]:
# @jax.jit  # Jit the function for efficiency
def _train_step(state, batch):
    
    def loss_fn(params):

        data, labels = batch
        print(data.shape)
        # Same as model.apply
        logits = state.apply_fn(
            {"params": params},
            data,
        )

        b, t, c = logits.shape
        logits = logits.reshape((b * t, c))
        labels = labels.reshape((b * t))
        labels_one_hot = nn.one_hot(labels, num_classes=c)


        loss = optax.losses.softmax_cross_entropy(logits=logits, labels=labels_one_hot)
        mean_loss = jnp.mean(loss)
        return mean_loss, logits

    # Gradient function
    grad_fn = jax.value_and_grad(
        loss_fn,  # Function to calculate the loss
        has_aux=True,  # Function has additional outputs, here accuracy
    )
    # Determine gradients for current model, parameters and batch
    (loss, logits), grads = grad_fn(state.params)
    # accuracy = jnp.mean(jnp.argmax(logits, -1) == labels)

    # Perform parameter update with gradients and optimizer
    state = state.apply_gradients(grads=grads)
    # Return state and any other value we might want
    return state, loss


@jax.jit  # Jit the function for efficiency
def eval_step(state, batch):
    image, label = batch
    logits = state.apply_fn({"params": state.params}, label)
    b, t, c = logits.shape
    logits = logits.reshape((b * t, c))
    labels = labels.reshape((b * t))
    labels_one_hot = nn.one_hot(labels, num_classes=tokenizer.get_vocab_size())

    loss = optax.losses.softmax_cross_entropy(logits=logits, labels=labels_one_hot)
    return loss


def train(state, num_epochs=100):
    
    for epoch in tqdm(range(num_epochs)):
        train_batch, labels_batch = batch_loader.get_batch(
            key, batch_size, context_lenght, is_train=True
        )

        batch = (train_batch, labels_batch)
        
        epoch_loss = jnp.array([])
        epoch_acc = jnp.array([])

        # for batch in batches:
        state, loss = _train_step(state, batch)

        jnp.append(epoch_loss, loss)
        # epoch_acc.append(acc)
        # We could use the loss and accuracy for logging here, e.g. in TensorBoard

        print(f"Loss at epoch {epoch}: {loss}")
        # print(f"Accuracy at epoch {epoch}: {epoch_acc.mean()}")
    return state

In [13]:
# Optimizer

scheduler = optax.warmup_cosine_decay_schedule(
    init_value=0.01, peak_value=1, warmup_steps=100, decay_steps=2000
)
optimizer = optax.adamw(scheduler)

In [14]:
# Training

model_state = train_state.TrainState.create(
    apply_fn=model.apply,
    params=params["params"],
    tx=optimizer,
)

trained_model_state = train(model_state, num_epochs=50)

  0%|          | 0/50 [00:00<?, ?it/s]

(4, 8)
Loss at epoch 0: 5.110974311828613
(4, 8)
Loss at epoch 1: 5.091230869293213
(4, 8)
Loss at epoch 2: 5.051956653594971
(4, 8)
Loss at epoch 3: 4.993185043334961
(4, 8)
Loss at epoch 4: 4.914970874786377
(4, 8)
Loss at epoch 5: 4.817394256591797
(4, 8)
Loss at epoch 6: 4.700573444366455
(4, 8)
Loss at epoch 7: 4.564676761627197
(4, 8)
Loss at epoch 8: 4.409941673278809
(4, 8)
Loss at epoch 9: 4.236703872680664
(4, 8)
Loss at epoch 10: 4.045437335968018
(4, 8)
Loss at epoch 11: 3.8368172645568848
(4, 8)
Loss at epoch 12: 3.6117992401123047
(4, 8)
Loss at epoch 13: 3.3717384338378906
(4, 8)
Loss at epoch 14: 3.118544816970825
(4, 8)
Loss at epoch 15: 2.8548762798309326
(4, 8)
Loss at epoch 16: 2.5843544006347656
(4, 8)
Loss at epoch 17: 2.3117523193359375
(4, 8)
Loss at epoch 18: 2.0430595874786377
(4, 8)
Loss at epoch 19: 1.785287618637085
(4, 8)
Loss at epoch 20: 1.5458985567092896
(4, 8)
Loss at epoch 21: 1.3318400382995605
(4, 8)
Loss at epoch 22: 1.1483559608459473
(4, 8)
Loss

In [15]:
# Generate after training

key, subkey = jax.random.split(key)
generated_seq = model.generate(
    key=subkey,
    params=trained_model_state.params,
    data=jax.numpy.zeros((1, 1), dtype=jax.numpy.int32),
    length=300,
)

decoded_text = tokenizer.decode(generated_seq[0])

print(decoded_text)


is embot embodis embot f embot guly not embodis f guly gulp embod gulp embot gulp embodis embot guly embof fly ndis nof emboflp embot f embot nod φήὖδ3πodis gulp f guly embot nd guly guly embot gulp fly f gulp emboflp ndis embot gulp embodis f embodis emboflp guly emboflp f guly nd fly not embot gul
